<a href="https://colab.research.google.com/github/LucasAlegre/vote-network/blob/master/vote_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --user graphistry
! pip install python-igraph
! pip install networkx

In [ ]:
! pip install networkx==2.6

     |████████████████████████████████| 1.9 MB 7.9 MB/s 
     |████████████████████████████████| 28.5 MB 30 kB/s 
     |████████████████████████████████| 10.3 MB 33.0 MB/s 
Reason for being yanked: Need to resolve: https://github.com/networkx/networkx/pull/4967
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: networkx
    Found existing installation: networkx 2.5.1
    Uninstalling networkx-2.5.1:
      Successfully uninstalled networkx-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is

In [ ]:
import graphistry
import networkx
    
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="", password="")

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving g.graphml to g.graphml


In [ ]:
g = networkx.read_graphml('g.graphml', )

In [ ]:
graphistry.bind(source='src', destination='dst', node='id').plot(g)